In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\geoguess\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\geoguess'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    


In [6]:
from AI_Vs_Human_Geoguess.constants import *
from AI_Vs_Human_Geoguess.utils.common import read_yaml,create_directories

In [7]:


    
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
import numpy as np
from AI_Vs_Human_Geoguess import logger
from AI_Vs_Human_Geoguess.utils.common import get_size,normalize_long,normalize_lat
import pandas as pd

In [27]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    def download_file(self)->str:
        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')
            file_id=dataset_url.split('/')[-2]
            prefix='http://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f'Download data from {dataset_url} into file {zip_download_dir}')
        except Exception as e:
            raise e
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    def preprocess_data(self):
        data=pd.read_csv(os.path.join(self.config.unzip_dir,'dataset\coords.csv'),header=None)
        data.rename(columns={0:'latitude',1:'longitude'},inplace=True)
        image=[]
        for i in (os.listdir(os.path.join(self.config.unzip_dir,'dataset'))):
            if i.endswith('.png'):
                image.append(i)
        
        data['image']=np.array(sorted(image,key=lambda x:int(x.split('.')[0])))
        data.rename(columns={0:'latitude',1:'longitude'},inplace=True)
        data['normalized_lat']=data['latitude'].apply(normalize_lat)
        data['normalized_long']=data['longitude'].apply(normalize_long)
        data.to_csv(os.path.join(self.config.unzip_dir,'dataset\coords.csv'))
        



   


In [28]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    dataingestion=DataIngestion(config=data_ingestion_config)
    dataingestion.download_file()
    dataingestion.extract_zip_file()
    dataingestion.preprocess_data()
except Exception as e:
    raise e

[2024-08-06 22:57:13,792: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-06 22:57:13,795: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-06 22:57:13,797: INFO: common: created directory at: artifacts]
hello
[2024-08-06 22:57:13,799: INFO: common: created directory at: artifacts/data_ingestion]
